In [2]:
# scrape wikipeida
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer,util
import torch
import faiss
import textwrap
from dotenv import load_dotenv
from tqdm import tqdm
import os
import pickle
import anthropic
from glob import glob
import numpy as np
load_dotenv()

/Users/blackbear/IChing/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


True

In [3]:
client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=os.environ["CLAUDE_API_KEY"],
)


let's get all the universities' alumni

In [10]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

/Users/blackbear/IChing/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


storing all the alumni's wiki pages

In [4]:

n_downloaded_lists = len(glob("data/*.obj"))

In [5]:
with open('manon-audebert.md','r') as file:
    input =  file.read()
    
message = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1000,
    temperature=0,
    system="I am trying to do a semantic comparison between two people's bio. The reference bio would be Wikipedia articles on people broken down into top-level sections (career, personal life, education, etc.). Users will input their bio with the details. What you will need to do is to rearrange the bio into similar sections, rewrite the bio to match Wikipedia style writing for each section, and add five astrisks after each section ***** (keep the title and the body of the section together in one section) .  You don't need to respond with pleasantry. \n ",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": input
                }
            ]
        }
    ]
)
sectioned = message.content[0].text.split("*****")
#slighed cleaning of the sections
cleaned_sections = [x for x in sectioned if len(x)>3]
with open('temp.pkl',"wb") as file:
    pickle.dump(cleaned_sections,file)

In [6]:
with open('temp.pkl','rb') as file:
    cleaned_sections= pickle.load(file)
cleaned_sections

['Early Life and Education\n\nManon Audebert pursued her undergraduate studies at Pomona College in Claremont, California, where she earned a Bachelor of Arts in Computer Science with a minor in Mathematics from 2014 to 2018. During her time at Pomona, Audebert demonstrated leadership skills as the captain of the NCAA Division III Pomona-Pitzer Varsity Swim and Dive team in her senior year, having been a member of the team throughout her college career.\n\nAudebert\'s academic interests extended beyond her coursework. In 2017, she presented a poster at the Grace Hopper Conference on "The Role of Gender in CS Classwork Collaboration and its Impact on Women\'s Success," showcasing her early engagement with issues of gender diversity in technology.\n\nFollowing her undergraduate studies, Audebert continued her education at Columbia University in New York City. She is currently pursuing two advanced degrees simultaneously: a Master of Business Administration with a focus on Technology and 

In [7]:
class EssaySearchEngine:
    def __init__(self, embedding_dim, device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.embedding_dim = embedding_dim
        self.device = device
        self.embeddings = None
        self.index = faiss.IndexFlatIP(self.embedding_dim)
        self.metadata = []  # To store any associated metadata with embeddings
        self.model=SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

    def batch_add_embedding_list(self, embedding_list, metadata=None,batch_size=32): 
        for i in tqdm(range(0, len(embedding_list), batch_size), desc="Adding content"):
            batch = embedding_list[i:i+batch_size]
            embeddings = batch.to(self.device)
            cpu_embeddings = embeddings.cpu().numpy().astype('float32')
            self.index.add(cpu_embeddings)
            
            if metadata:
                self.metadata.extend(metadata[i:i+batch_size])
            else:
                self.metadata.extend(range(i, min(i+batch_size, len(embedding_list))))         

    def search(self, content, k=5):
        query_embedding = self.model.encode(content, convert_to_tensor=True)
        query_tensor = torch.tensor(query_embedding, device=self.device).unsqueeze(0)
        
        # Convert query to CPU and numpy for FAISS
        cpu_query = query_tensor.cpu().numpy().astype('float32')
        
        D, I = self.index.search(cpu_query, k)
        print(D,I)
        results = [(self.metadata[i] if self.metadata else i, score) for i, score in zip(I[0], D[0])]
        return results
    
    def batch_search(self, content_list, k=5, batch_size=32):
        all_results = []
        for i in tqdm(range(0, len(content_list), batch_size), desc="Searching"):
            batch = content_list[i:i+batch_size]
            query_embeddings = self.model.encode(batch, convert_to_tensor=True, device=self.device)
            batch_results = self._batch_search_embeddings(query_embeddings, k)
            all_results.extend(batch_results)
        return all_results
    
    def _batch_search_embeddings(self, query_embeddings, k):
        cpu_queries = query_embeddings.cpu().numpy().astype('float32')
        if cpu_queries.ndim == 1:
            cpu_queries = cpu_queries.reshape(1, -1)
        D, I = self.index.search(cpu_queries, k)
        
        results = []
        for indices, distances in zip(I, D):
            result = [(self.metadata[i] if self.metadata else i, score) for i, score in zip(indices, distances)]
            results.append(result)
        
        return results

In [8]:

file_path ="data/*.obj" 
device='cuda' if torch.cuda.is_available() else 'cpu'
embeddings= []
meta_data=[]
files = glob(file_path)

index = faiss.IndexFlatIP(384)

for file in files:
    meta_data_list = []
    with open(file,'rb') as file:
        objs= pickle.load(file)
        for o in objs:     
            n_sections=len(o['sections_embedded_tensor'])
            obj_meta_data = [o['wiki_page']]*n_sections
            cpu_embeddings = o['sections_embedded_tensor'].to(device).cpu().numpy().astype('float32')
            embeddings.extend(cpu_embeddings)
            if len(cpu_embeddings)== 0: continue
            index.add(cpu_embeddings)
            meta_data.extend(obj_meta_data)

        
len(embeddings)


146762

In [11]:
results = []
for section in cleaned_sections:
    cpu_queries = model.encode([section],convert_to_tensor=True,device=device).cpu().numpy().astype('float32')
    D, I = index.search(cpu_queries, 2)
    results.extend([[(meta_data[i] if meta_data else i, score) for i, score in zip(indices, distances)] 
                   for indices, distances in zip(I, D)])


In [12]:
cleaned_sections

['Early Life and Education\n\nManon Audebert pursued her undergraduate studies at Pomona College in Claremont, California, where she earned a Bachelor of Arts in Computer Science with a minor in Mathematics from 2014 to 2018. During her time at Pomona, Audebert demonstrated leadership skills as the captain of the NCAA Division III Pomona-Pitzer Varsity Swim and Dive team in her senior year, having been a member of the team throughout her college career.\n\nAudebert\'s academic interests extended beyond her coursework. In 2017, she presented a poster at the Grace Hopper Conference on "The Role of Gender in CS Classwork Collaboration and its Impact on Women\'s Success," showcasing her early engagement with issues of gender diversity in technology.\n\nFollowing her undergraduate studies, Audebert continued her education at Columbia University in New York City. She is currently pursuing two advanced degrees simultaneously: a Master of Business Administration with a focus on Technology and 

In [13]:
results

[[(AnitaB.org (id: 32234869, ns: 0), 0.5434913),
  (Esther Brimmer (id: 3851380, ns: 0), 0.5392928)],
 [(Alteryx (id: 44133735, ns: 0), 0.43991828),
  (Google (id: 1092923, ns: 0), 0.43822727)],
 [(Maria Klawe (id: 2976395, ns: 0), 0.45175576),
  (Shannon Vallor (id: 53394084, ns: 0), 0.42521742)],
 [(Albert Einstein World Award of Science (id: 416820, ns: 0), 0.4937371),
  (Daniel Lubetzky (id: 34857443, ns: 0), 0.49162686)],
 [(Margaret A. Tolbert (id: 75643084, ns: 0), 0.475733),
  (Nico Young (id: 68822842, ns: 0), 0.47561672)]]

In [19]:
results[4][1][0].sections

[Section: Athletic career (1):
 
 Subsections (2):
 Section: High school career (2):
 Young ran for Newbury Park High School in Newbury Park, California. He broke the 3 mile cross country American high school record at the 2019 Woodbridge Invitational, running a time of 13:39.7, and lowered the American high school indoor 3000m record (previously held by Drew Hunter), running a time of 7:56.91. At the 2019 Nike Cross Nationals, Young won the race and set the course record with a time of 14:52 and led Newbury Park to the team victory. Young was named the 2019-20 XC National Gatorade Player of the Year and the 2019-20 Track and Field National Gatorade Player of the Year.  He was the first to sweep both awards in the same season. His brother Leo picked up the California version of the Cross Country award in 2022.  His senior season in 2020 was wiped out due to the COVID-19 pandemic.  His season amounted to time trials with his brothers and teammates.
 Subsections (0):
 
 Section: Collegia